# 📚 01. 데이터 처리 및 전처리

## 목표
- 꿀스테이 7개 도메인 마크다운 파일 로딩
- 문서 구조 분석 및 파싱
- 청킹 전략 실험
- 메타데이터 추출

## 1. 환경 설정 및 라이브러리 임포트

In [10]:
import os
import re
from pathlib import Path
from typing import List, Dict, Any
from dataclasses import dataclass

from langchain.schema import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


## 2. 도메인 설정 및 파일 경로 정의

In [11]:
# 도메인별 설정
DOMAIN_CONFIG = {
    "hr_policy": {
        "file": "HR_Policy_Guide.md",
        "description": "인사정책, 근무시간, 휴가, 급여, 복리후생",
        "keywords": ["근무시간", "휴가", "급여", "복리후생", "인사", "채용", "평가"]
    },
    "tech_policy": {
        "file": "Tech_Policy_Guide.md",
        "description": "기술정책, 개발환경, 코딩표준, 보안정책",
        "keywords": ["개발", "기술", "코딩", "보안", "테스트", "배포", "인프라"]
    },
    "architecture": {
        "file": "Architecture_Guide.md",
        "description": "CMS 아키텍처, 시스템설계, 레이어구조",
        "keywords": ["아키텍처", "시스템", "설계", "구조", "레이어", "모듈"]
    },
    "component": {
        "file": "Component_Guide.md",
        "description": "컴포넌트 가이드라인, UI/UX 표준",
        "keywords": ["컴포넌트", "UI", "UX", "디자인", "인터페이스", "사용자"]
    },
    "deployment": {
        "file": "Deployment_Guide.md",
        "description": "배포프로세스, CI/CD, 환경관리",
        "keywords": ["배포", "CI/CD", "환경", "빌드", "릴리스", "운영"]
    },
    "development": {
        "file": "Development_Process_Guide.md",
        "description": "개발프로세스, 워크플로우, 협업규칙",
        "keywords": ["프로세스", "워크플로우", "협업", "스프린트", "애자일"]
    },
    "business_policy": {
        "file": "Business_Policy_Guide.md",
        "description": "비즈니스정책, 운영규칙, 의사결정",
        "keywords": ["비즈니스", "정책", "운영", "의사결정", "전략"]
    }
}

# 데이터 경로 설정
DATA_DIR = Path("../data")
print(f"📁 데이터 디렉토리: {DATA_DIR.absolute()}")
print(f"📊 총 {len(DOMAIN_CONFIG)}개 도메인 설정 완료")

📁 데이터 디렉토리: /Users/yundoun/Desktop/project/legal_rag/coolstay_rag/notebooks/../data
📊 총 7개 도메인 설정 완료


## 3. 문서 로딩 및 기본 분석

In [12]:
def load_markdown_files() -> Dict[str, str]:
    """모든 마크다운 파일을 로딩합니다."""
    documents = {}
    
    for domain, config in DOMAIN_CONFIG.items():
        file_path = DATA_DIR / config["file"]
        
        if file_path.exists():
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                documents[domain] = content
                print(f"✅ {domain}: {len(content):,}자, {len(content.splitlines())}줄")
        else:
            print(f"❌ {domain}: 파일 없음 - {file_path}")
    
    return documents

# 문서 로딩
raw_documents = load_markdown_files()
print(f"\n📚 총 {len(raw_documents)}개 문서 로딩 완료")

✅ hr_policy: 1,782자, 93줄
✅ tech_policy: 3,889자, 172줄
✅ architecture: 12,779자, 542줄
✅ component: 7,689자, 423줄
✅ deployment: 5,436자, 306줄
✅ development: 3,562자, 181줄
✅ business_policy: 2,243자, 110줄

📚 총 7개 문서 로딩 완료


## 4. 마크다운 구조 분석

In [13]:
def analyze_markdown_structure(content: str, domain: str) -> Dict[str, Any]:
    """마크다운 문서의 구조를 분석합니다."""
    lines = content.split('\n')
    
    # 헤더 구조 분석
    headers = {
        'h1': [],
        'h2': [],
        'h3': [],
        'h4': []
    }
    
    for i, line in enumerate(lines):
        if line.startswith('# '):
            headers['h1'].append((i, line.strip('# ').strip()))
        elif line.startswith('## '):
            headers['h2'].append((i, line.strip('# ').strip()))
        elif line.startswith('### '):
            headers['h3'].append((i, line.strip('# ').strip()))
        elif line.startswith('#### '):
            headers['h4'].append((i, line.strip('# ').strip()))
    
    # 코드 블록 및 특수 구조 분석
    code_blocks = len(re.findall(r'```[\s\S]*?```', content))
    bullet_points = len(re.findall(r'^[-*+]\s', content, re.MULTILINE))
    numbered_lists = len(re.findall(r'^\d+\.\s', content, re.MULTILINE))
    
    return {
        'domain': domain,
        'total_lines': len(lines),
        'total_chars': len(content),
        'headers': headers,
        'code_blocks': code_blocks,
        'bullet_points': bullet_points,
        'numbered_lists': numbered_lists
    }

# 각 문서 구조 분석
document_structures = {}
for domain, content in raw_documents.items():
    structure = analyze_markdown_structure(content, domain)
    document_structures[domain] = structure
    
    print(f"\n📋 {domain.upper()} 구조 분석:")
    print(f"  - 총 라인: {structure['total_lines']:,}줄")
    print(f"  - 총 문자: {structure['total_chars']:,}자")
    print(f"  - H1 헤더: {len(structure['headers']['h1'])}개")
    print(f"  - H2 헤더: {len(structure['headers']['h2'])}개")
    print(f"  - H3 헤더: {len(structure['headers']['h3'])}개")
    print(f"  - 코드 블록: {structure['code_blocks']}개")
    print(f"  - 불릿 포인트: {structure['bullet_points']}개")


📋 HR_POLICY 구조 분석:
  - 총 라인: 93줄
  - 총 문자: 1,782자
  - H1 헤더: 1개
  - H2 헤더: 7개
  - H3 헤더: 13개
  - 코드 블록: 0개
  - 불릿 포인트: 47개

📋 TECH_POLICY 구조 분석:
  - 총 라인: 172줄
  - 총 문자: 3,889자
  - H1 헤더: 1개
  - H2 헤더: 9개
  - H3 헤더: 22개
  - 코드 블록: 3개
  - 불릿 포인트: 76개

📋 ARCHITECTURE 구조 분석:
  - 총 라인: 542줄
  - 총 문자: 12,779자
  - H1 헤더: 1개
  - H2 헤더: 11개
  - H3 헤더: 22개
  - 코드 블록: 22개
  - 불릿 포인트: 0개

📋 COMPONENT 구조 분석:
  - 총 라인: 423줄
  - 총 문자: 7,689자
  - H1 헤더: 1개
  - H2 헤더: 11개
  - H3 헤더: 25개
  - 코드 블록: 22개
  - 불릿 포인트: 17개

📋 DEPLOYMENT 구조 분석:
  - 총 라인: 306줄
  - 총 문자: 5,436자
  - H1 헤더: 19개
  - H2 헤더: 10개
  - H3 헤더: 23개
  - 코드 블록: 11개
  - 불릿 포인트: 47개

📋 DEVELOPMENT 구조 분석:
  - 총 라인: 181줄
  - 총 문자: 3,562자
  - H1 헤더: 1개
  - H2 헤더: 8개
  - H3 헤더: 19개
  - 코드 블록: 1개
  - 불릿 포인트: 74개

📋 BUSINESS_POLICY 구조 분석:
  - 총 라인: 110줄
  - 총 문자: 2,243자
  - H1 헤더: 1개
  - H2 헤더: 7개
  - H3 헤더: 15개
  - 코드 블록: 0개
  - 불릿 포인트: 60개


## 5. 청킹 전략 실험

In [14]:
def create_chunks_by_headers(content: str, domain: str) -> List[Document]:
    """헤더 기반으로 청킹합니다."""
    
    # 헤더 기반 분할
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )
    
    md_header_splits = markdown_splitter.split_text(content)
    
    # 메타데이터 강화
    enhanced_docs = []
    for doc in md_header_splits:
        # 기본 메타데이터에 도메인 정보 추가
        doc.metadata.update({
            "domain": domain,
            "description": DOMAIN_CONFIG[domain]["description"],
            "keywords": DOMAIN_CONFIG[domain]["keywords"],
            "chunk_type": "header_based"
        })
        enhanced_docs.append(doc)
    
    return enhanced_docs

def create_chunks_by_size(content: str, domain: str, chunk_size: int = 1000) -> List[Document]:
    """고정 크기 기반으로 청킹합니다."""
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    chunks = text_splitter.split_text(content)
    
    # Document 객체로 변환하며 메타데이터 추가
    documents = []
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk,
            metadata={
                "domain": domain,
                "description": DOMAIN_CONFIG[domain]["description"],
                "keywords": DOMAIN_CONFIG[domain]["keywords"],
                "chunk_type": "size_based",
                "chunk_index": i,
                "chunk_size": len(chunk)
            }
        )
        documents.append(doc)
    
    return documents

# 청킹 실험
chunking_results = {}

for domain, content in raw_documents.items():
    print(f"\n🔄 {domain} 청킹 실험 중...")
    
    # 헤더 기반 청킹
    header_chunks = create_chunks_by_headers(content, domain)
    
    # 크기 기반 청킹 (여러 크기 실험)
    size_chunks_1000 = create_chunks_by_size(content, domain, 1000)
    size_chunks_1500 = create_chunks_by_size(content, domain, 1500)
    
    chunking_results[domain] = {
        "header_based": header_chunks,
        "size_1000": size_chunks_1000,
        "size_1500": size_chunks_1500
    }
    
    print(f"  ✅ 헤더 기반: {len(header_chunks)}개 청크")
    print(f"  ✅ 크기 1000: {len(size_chunks_1000)}개 청크")
    print(f"  ✅ 크기 1500: {len(size_chunks_1500)}개 청크")

print(f"\n🎯 청킹 실험 완료 - 총 {len(chunking_results)}개 도메인")


🔄 hr_policy 청킹 실험 중...
  ✅ 헤더 기반: 14개 청크
  ✅ 크기 1000: 3개 청크
  ✅ 크기 1500: 2개 청크

🔄 tech_policy 청킹 실험 중...
  ✅ 헤더 기반: 23개 청크
  ✅ 크기 1000: 5개 청크
  ✅ 크기 1500: 3개 청크

🔄 architecture 청킹 실험 중...
  ✅ 헤더 기반: 23개 청크
  ✅ 크기 1000: 17개 청크
  ✅ 크기 1500: 11개 청크

🔄 component 청킹 실험 중...
  ✅ 헤더 기반: 26개 청크
  ✅ 크기 1000: 11개 청크
  ✅ 크기 1500: 6개 청크

🔄 deployment 청킹 실험 중...
  ✅ 헤더 기반: 24개 청크
  ✅ 크기 1000: 7개 청크
  ✅ 크기 1500: 5개 청크

🔄 development 청킹 실험 중...
  ✅ 헤더 기반: 20개 청크
  ✅ 크기 1000: 5개 청크
  ✅ 크기 1500: 3개 청크

🔄 business_policy 청킹 실험 중...
  ✅ 헤더 기반: 16개 청크
  ✅ 크기 1000: 3개 청크
  ✅ 크기 1500: 2개 청크

🎯 청킹 실험 완료 - 총 7개 도메인


## 6. 청킹 품질 분석

In [15]:
def analyze_chunk_quality(chunks: List[Document], strategy_name: str) -> Dict[str, Any]:
    """청크 품질을 분석합니다."""
    if not chunks:
        return {}
    
    chunk_sizes = [len(doc.page_content) for doc in chunks]
    
    return {
        "strategy": strategy_name,
        "total_chunks": len(chunks),
        "avg_chunk_size": sum(chunk_sizes) / len(chunk_sizes),
        "min_chunk_size": min(chunk_sizes),
        "max_chunk_size": max(chunk_sizes),
        "total_content_size": sum(chunk_sizes)
    }

# 청킹 품질 분석
print("📊 청킹 전략별 품질 분석\n")
print(f"{'도메인':<15} {'전략':<12} {'청크수':<8} {'평균크기':<10} {'최소크기':<10} {'최대크기':<10}")
print("-" * 80)

quality_analysis = {}
for domain, strategies in chunking_results.items():
    quality_analysis[domain] = {}
    
    for strategy_name, chunks in strategies.items():
        quality = analyze_chunk_quality(chunks, strategy_name)
        quality_analysis[domain][strategy_name] = quality
        
        if quality:  # 빈 결과가 아닌 경우만 출력
            print(f"{domain:<15} {strategy_name:<12} {quality['total_chunks']:<8} "
                  f"{quality['avg_chunk_size']:<10.0f} {quality['min_chunk_size']:<10} {quality['max_chunk_size']:<10}")

📊 청킹 전략별 품질 분석

도메인             전략           청크수      평균크기       최소크기       최대크기      
--------------------------------------------------------------------------------
hr_policy       header_based 14       108        33         167       
hr_policy       size_1000    3        698        242        941       
hr_policy       size_1500    2        956        472        1440      
tech_policy     header_based 23       148        34         321       
tech_policy     size_1000    5        903        774        967       
tech_policy     size_1500    3        1421       1335       1493      
architecture    header_based 23       475        36         1245      
architecture    size_1000    17       838        482        992       
architecture    size_1500    11       1224       867        1495      
component       header_based 26       245        37         429       
component       size_1000    11       824        314        995       
component       size_1500    6        1359       91

## 7. 최적 청킹 전략 선택

In [16]:
def select_best_chunking_strategy(quality_analysis: Dict) -> Dict[str, str]:
    """도메인별 최적 청킹 전략을 선택합니다."""
    best_strategies = {}
    
    for domain, strategies in quality_analysis.items():
        # 평가 기준: 적절한 청크 수 + 균형잡힌 크기
        # 너무 많은 청크는 검색 성능 저하, 너무 적으면 정보 손실
        
        best_score = float('inf')
        best_strategy = "header_based"  # 기본값
        
        for strategy_name, quality in strategies.items():
            if not quality:
                continue
                
            # 점수 계산 (낮을수록 좋음)
            # 청크 수가 너무 많거나 적으면 패널티
            chunk_count_penalty = abs(quality['total_chunks'] - 10) * 0.5
            
            # 평균 크기가 800-1200 범위에서 벗어나면 패널티
            avg_size = quality['avg_chunk_size']
            size_penalty = max(0, abs(avg_size - 1000) - 200) * 0.01
            
            # 크기 편차가 클수록 패널티
            size_variance = quality['max_chunk_size'] - quality['min_chunk_size']
            variance_penalty = size_variance * 0.001
            
            total_score = chunk_count_penalty + size_penalty + variance_penalty
            
            if total_score < best_score:
                best_score = total_score
                best_strategy = strategy_name
        
        best_strategies[domain] = best_strategy
        print(f"🎯 {domain}: {best_strategy} (점수: {best_score:.2f})")
    
    return best_strategies

# 최적 전략 선택
best_chunking_strategies = select_best_chunking_strategy(quality_analysis)

# 선택된 전략으로 최종 문서 준비
final_documents = {}
for domain, strategy in best_chunking_strategies.items():
    final_documents[domain] = chunking_results[domain][strategy]
    print(f"✅ {domain}: {len(final_documents[domain])}개 청크 준비 완료")

🎯 hr_policy: size_1500 (점수: 4.97)
🎯 tech_policy: size_1000 (점수: 2.69)
🎯 architecture: size_1500 (점수: 1.37)
🎯 component: size_1000 (점수: 1.18)
🎯 deployment: size_1000 (점수: 1.78)
🎯 development: size_1000 (점수: 3.06)
🎯 business_policy: size_1000 (점수: 3.90)
✅ hr_policy: 2개 청크 준비 완료
✅ tech_policy: 5개 청크 준비 완료
✅ architecture: 11개 청크 준비 완료
✅ component: 11개 청크 준비 완료
✅ deployment: 7개 청크 준비 완료
✅ development: 5개 청크 준비 완료
✅ business_policy: 3개 청크 준비 완료


## 8. 결과 요약 및 저장

In [17]:
# 처리 결과 요약
print("\n" + "="*60)
print("📋 데이터 처리 결과 요약")
print("="*60)

total_chunks = 0
total_content_size = 0

for domain, chunks in final_documents.items():
    chunk_count = len(chunks)
    content_size = sum(len(doc.page_content) for doc in chunks)
    avg_size = content_size / chunk_count if chunk_count > 0 else 0
    
    total_chunks += chunk_count
    total_content_size += content_size
    
    print(f"{domain:<15}: {chunk_count:>3}개 청크, 평균 {avg_size:>4.0f}자")

print("-" * 60)
print(f"{'총계':<15}: {total_chunks:>3}개 청크, 총 {total_content_size:>7,}자")
print(f"{'평균':<15}: {total_content_size/total_chunks:>7.0f}자/청크")

# 다음 노트북을 위한 데이터 저장 준비
print(f"\n🎯 다음 단계: 02_vector_stores.ipynb에서 벡터 저장소 구축")
print(f"📊 준비된 데이터: {len(final_documents)}개 도메인, {total_chunks}개 청크")


📋 데이터 처리 결과 요약
hr_policy      :   2개 청크, 평균  956자
tech_policy    :   5개 청크, 평균  903자
architecture   :  11개 청크, 평균 1224자
component      :  11개 청크, 평균  824자
deployment     :   7개 청크, 평균  896자
development    :   5개 청크, 평균  838자
business_policy:   3개 청크, 평균  838자
------------------------------------------------------------
총계             :  44개 청크, 총  41,929자
평균             :     953자/청크

🎯 다음 단계: 02_vector_stores.ipynb에서 벡터 저장소 구축
📊 준비된 데이터: 7개 도메인, 44개 청크


## 9. 샘플 청크 확인

In [18]:
# 각 도메인별 첫 번째 청크 샘플 출력
print("\n📋 도메인별 청크 샘플 (첫 300자)\n")

for domain, chunks in final_documents.items():
    if chunks:
        first_chunk = chunks[0]
        sample_content = first_chunk.page_content[:300] + "..." if len(first_chunk.page_content) > 300 else first_chunk.page_content
        
        print(f"🏷️  {domain.upper()}")
        print(f"📄 메타데이터: {first_chunk.metadata}")
        print(f"📝 내용 샘플: {sample_content}")
        print("-" * 80)

print("\n✅ 01_data_processing.ipynb 완료!")
print("🚀 다음: 02_vector_stores.ipynb 실행")


📋 도메인별 청크 샘플 (첫 300자)

🏷️  HR_POLICY
📄 메타데이터: {'domain': 'hr_policy', 'description': '인사정책, 근무시간, 휴가, 급여, 복리후생', 'keywords': ['근무시간', '휴가', '급여', '복리후생', '인사', '채용', '평가'], 'chunk_type': 'size_based', 'chunk_index': 0, 'chunk_size': 1440}
📝 내용 샘플: # 꿀스테이 인사정책 가이드

## 📋 개요
꿀스테이 직원들을 위한 인사정책 및 근무 규정을 안내합니다.

## 🕐 근무시간 및 휴가

### 정규 근무시간
- **평일**: 오전 9:00 ~ 오후 6:00 (점심시간 12:00~13:00 제외)
- **주 40시간** 근무제 운영
- **코어타임**: 오전 10:00 ~ 오후 4:00 (필수 근무시간)

### 유연근무제
- **시차출퇴근**: 오전 8:00~10:00 출근, 오후 5:00~7:00 퇴근 가능
- **재택근무**: 주 2일까지 가능 (사전 승인 필요)
- **하이브...
--------------------------------------------------------------------------------
🏷️  TECH_POLICY
📄 메타데이터: {'domain': 'tech_policy', 'description': '기술정책, 개발환경, 코딩표준, 보안정책', 'keywords': ['개발', '기술', '코딩', '보안', '테스트', '배포', '인프라'], 'chunk_type': 'size_based', 'chunk_index': 0, 'chunk_size': 774}
📝 내용 샘플: # 꿀스테이 기술정책 가이드

## 📋 개요
꿀스테이 개발팀의 기술 정책 및 개발 가이드라인을 안내합니다.

## 🏗️ 개발 환경 및 인프라

### 개발 환경 구성
- **로컬 개발**: Node.js 18+, React 18, Material-UI v